# Result Comparison

In this notebook the results of the FC and the TN approach are compared against each other

### imports

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
import re

### import data

In [ ]:
results_FC = pd.read_csv('results/FC_test_results.csv')
#drop Unnamed: 0
results_FC = results_FC.drop(columns=['Unnamed: 0', 'idx'])
# add FC as column
results_FC['model'] = 'FC'

results_TN = pd.read_csv('results/TN_test_results.csv')
#drop Unnamed: 0
results_TN = results_TN.drop(columns=['Unnamed: 0', 'idx'])
# add TN as column
results_TN['model'] = 'TN'

results = pd.concat([results_FC, results_TN])
results.sample(5)

,points_transformed,positions,mse,subset,video,clip,frame,points,model
81,[[159.6 78. ]\n [183.6 78. ]...,[[157.60052 74.256226]\n [181.969 74.109...,54.839863,CourtKeypoints,NjWIlLiEt94,Clip1,100,"{'top_left_corner': [399, 195], 'top_left_sing...",FC
16,[[169.6 115.200005]\n [188.8 115.200...,[[ 141.75636 124.49932 ]\n [ 166.7745 124...,1451.800800,New,Video_6,clip_1,200,"{'top_left_corner': [424, 288], 'top_left_sing...",FC
15,[[169.6 115.200005]\n [188.8 115.200...,"[(172, 114), (191, 114), (301, 115), (323, 116...",3.698744,New,Video_6,clip_1,150,"{'top_left_corner': [424, 288], 'top_left_sing...",TN
64,[[172.8 65.200005]\n [194. 64.8 ...,"[(172, 64), (193, 63), (318, 64), (340, 64), (...",2.229535,CourtKeypoints,FIemQkUaM7A,Clip1,1050,"{'top_left_corner': [432, 163], 'top_left_sing...",TN
69,[[156.8 76. ]\n [181.6 76. ...,[[145.47182 77.35306 ]\n [172.9377 77.038...,58.190678,CourtKeypoints,GFpd2pk6H88,Clip1,1700,"{'top_left_corner': [392, 190], 'top_left_sing...",FC


In [10]:
def clean_and_convert(string):
    if isinstance(string, str):  # Check if the input is a string
        # Replace spaces with commas, remove duplicates, and ensure valid list format
        string = string.replace("\n", ",").replace(" ", ",")
        string = re.sub(r",+", ",", string)  # Replace multiple commas with a single comma
        string = string.replace("[,", "[").replace(",]", "]")  # Remove leading/trailing commas
        try:
            return ast.literal_eval(string)  # Safely evaluate to a list
        except Exception as e:
            print(f"Error processing string: {string}, error: {e}")
            return None
    return string  # If already processed, return as-is

# convert points_tranformed to list of lists instead of string
results["points_transformed"] = results["points_transformed"].apply(clean_and_convert)
results["positions"] = results["positions"].apply(clean_and_convert)

results.sample(5)

,points_transformed,positions,mse,subset,video,clip,frame,points,model
135,"[[161.2, 84.0], [184.8, 84.0], [324.80002, 84....","[(161, 83), (184, 83), (324, 83), (348, 83), (...",1.984294,CourtKeypoints,a0MPorVoT_k,Clip1,100,"{'top_left_corner': [403, 210], 'top_left_sing...",TN
80,"[[161.2, 77.6], [185.2, 77.6], [326.80002, 77....","[(160, 78), (185, 78), (326, 78), (350, 78), (...",1.288271,CourtKeypoints,NjWIlLiEt94,Clip1,700,"{'top_left_corner': [403, 194], 'top_left_sing...",TN
185,"[[175.2, 57.600002], [195.6, 57.600002], [315....","[(175, 56), (195, 56), (314, 56), (335, 57), (...",2.429422,CourtKeypoints,xGYkRnpL-R8,Clip1,1650,"{'top_left_corner': [438, 144], 'top_left_sing...",TN
192,"[[144.40001, 81.200005], [169.6, 81.200005], [...","[[131.46912, 76.87885], [158.35, 76.71496], [3...",102.300760,CourtKeypoints,yWhxLb5bWJw,Clip1,600,"{'top_left_corner': [361, 203], 'top_left_sing...",FC
46,"[[159.6, 72.8], [184.0, 72.8], [328.0, 72.8], ...","[[154.42473, 72.06613], [179.43417, 71.924324]...",19.550620,CourtKeypoints,91qZ6E3gQkA,Clip1,1550,"{'top_left_corner': [399, 182], 'top_left_sing...",FC


## analyze results

In [12]:
def compute_mse(points, positions, img_size):
    #loop over points
    mse = []

    for i in range(len(points)):
        # get the point
        point = points[i]
        # get the position
        position = positions[i]

        # if the point is outside the image, skip
        if point[0] < 0 or point[0] >= img_size[0] or point[1] < 0 or point[1] >= img_size[1]:
            continue

        # if the position is -1,-1 return the maximum distance
        if position[0] == -1 and position[1] == -1:
            continue
        
        # calculate the distance
        distance = np.linalg.norm(np.array(point) - np.array(position))

        # add to mse
        mse.append(distance**2)

    # return the mean
    return np.mean(mse)

def compute_counts(points, positions, img_size, threshold=4):
    # check that len is the same
    assert len(points) == len(positions) 
    
    #loop over points
    tp = 0
    fp = 0
    fn = 0
    tn = 0

    for i in range(len(points)):
        # get the point
        point = points[i]
        # get the position
        position = positions[i]
        
        # calculate the distance
        distance = np.linalg.norm(np.array(point) - np.array(position))

        # add to tp, fp, fn, tn
        if distance <= threshold:
            # if point is outside the frame
            if point[0] < 0 or point[0] >= img_size[0] or point[1] < 0 or point[1] >= img_size[1]:
                tn += 1
            else:
                tp += 1
        else:
            #if point is outside the frame
            if point[0] < 0 or point[0] >= img_size[0] or point[1] < 0 or point[1] >= img_size[1]:
                fn += 1
            else:
                fp += 1

    # return the metrics
    return tp, fp, tn, fn


def calculate_metrics(tp, fp, tn, fn):
    # Avoid division by zero for precision and recall
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0

    # F1-score calculation
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    # Accuracy calculation
    total = tp + fp + tn + fn
    accuracy = (tp + tn) / total if total > 0 else 0.0

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [15]:
img_size = (512, 288)
# calculate scores for each model
results['mse'] = results.apply(lambda x: compute_mse(x['points_transformed'], x['positions'], img_size), axis=1)
results[['tp', 'fp', 'tn', 'fn']] = results.apply(lambda x: pd.Series(compute_counts(x['points_transformed'], x['positions'], img_size)), axis=1)
results.sample(5)

,points_transformed,positions,mse,subset,video,clip,frame,points,model,tp,fp,tn,fn
171,"[[169.2, 66.8], [204.0, 67.200005], [413.2, 70...","[(169, 65), (204, 66), (412, 68), (448, 68), (...",3.402507,CourtKeypoints,pnlmE-Iit4Q,Clip1,300,"{'top_left_corner': [423, 167], 'top_left_sing...",TN,13,1,0,1
93,"[[156.8, 80.0], [181.6, 80.0], [329.6, 80.0], ...","[[148.06882, 74.114395], [174.31433, 73.86227]...",86.565609,CourtKeypoints,RNqK9n4Q4bg,Clip1,2450,"{'top_left_corner': [392, 200], 'top_left_sing...",FC,1,14,0,0
109,"[[165.6, 77.200005], [188.40001, 77.200005], [...","[[157.38235, 75.44025], [181.86317, 75.20202],...",45.817549,CourtKeypoints,V6NR0NAN_cA,Clip1,2900,"{'top_left_corner': [414, 193], 'top_left_sing...",FC,3,12,0,0
19,"[[164.8, 66.4], [188.0, 66.4], [326.0, 66.4], ...","[[159.00656, 65.3898], [182.44029, 65.31711], ...",44.414401,CourtKeypoints,-34oYneHVGg,Clip1,2700,"{'top_left_corner': [412, 166], 'top_left_sing...",FC,4,11,0,0
44,"[[158.0, 66.4], [182.0, 66.4], [326.0, 66.4], ...","[(157, 66), (181, 66), (325, 65), (349, 65), (...",2.508225,CourtKeypoints,7qCfURaFMpQ,Clip1,800,"{'top_left_corner': [395, 166], 'top_left_sing...",TN,15,0,0,0


In [41]:
# compute total metrics 
metrics = results.groupby('model')[['tp', 'fp', 'tn', 'fn']].sum().reset_index()
#mse
metrics['mse'] = results.groupby('model')['mse'].mean().values
# calculate metrics
metrics[['accuracy', 'precision', 'recall', 'f1']] = metrics.apply(lambda x: pd.Series(calculate_metrics(x['tp'], x['fp'], x['tn'], x['fn'])), axis=1)
# add subset column total and remove tp, fp, tn, fn
metrics['subset'] = 'total'
metrics

,model,tp,fp,tn,fn,mse,accuracy,precision,recall,f1,subset
0,FC,768,2178,1,38,221.212804,0.257621,0.260692,0.952854,0.409382,total
1,TN,2904,42,0,39,7.912210,0.972864,0.985743,0.986748,0.986246,total


In [42]:
# compute total metrics for each subset and model
metrics_subset = results.groupby(['model', 'subset'])[['tp', 'fp', 'tn', 'fn']].sum().reset_index()
#mse
metrics_subset['mse'] = results.groupby(['model', 'subset'])['mse'].mean().values
# calculate metrics
metrics_subset[['accuracy', 'precision', 'recall', 'f1']] = metrics_subset.apply(lambda x: pd.Series(calculate_metrics(x['tp'], x['fp'], x['tn'], x['fn'])), axis=1)
metrics_subset

,model,subset,tp,fp,tn,fn,mse,accuracy,precision,recall,f1
0,FC,CourtKeypoints,742,1957,1,15,107.462715,0.273665,0.274917,0.980185,0.429398
1,FC,New,26,221,0,23,1365.033147,0.096296,0.105263,0.530612,0.175676
2,TN,CourtKeypoints,2665,34,0,16,8.251438,0.981584,0.987403,0.994032,0.990706
3,TN,New,239,8,0,23,4.501082,0.885185,0.967611,0.912214,0.939096


In [43]:
# combine
metrics_table = pd.concat([metrics, metrics_subset])
# sort by model and subset
metrics_table = metrics_table.sort_values(by=['model', 'subset'])
metrics_table

,model,tp,fp,tn,fn,mse,accuracy,precision,recall,f1,subset
0,FC,742,1957,1,15,107.462715,0.273665,0.274917,0.980185,0.429398,CourtKeypoints
1,FC,26,221,0,23,1365.033147,0.096296,0.105263,0.530612,0.175676,New
0,FC,768,2178,1,38,221.212804,0.257621,0.260692,0.952854,0.409382,total
2,TN,2665,34,0,16,8.251438,0.981584,0.987403,0.994032,0.990706,CourtKeypoints
3,TN,239,8,0,23,4.501082,0.885185,0.967611,0.912214,0.939096,New
1,TN,2904,42,0,39,7.912210,0.972864,0.985743,0.986748,0.986246,total


In [23]:
# indexes for botttom half
bottom_point_indexes = [4,5,6,7,10,11,13]
top_point_indexes = [0,1,2,3,8,9,12]
center_point_indexes = [14]

# create a new columnf with just the points with the indexes
results['bottom_predictions'] = results['positions'].apply(lambda x: [x[i] for i in bottom_point_indexes])
results['top_predictions'] = results['positions'].apply(lambda x: [x[i] for i in top_point_indexes])
results['center_predictions'] = results['positions'].apply(lambda x: [x[i] for i in center_point_indexes])

results['bottom_actual'] = results['points_transformed'].apply(lambda x: [x[i] for i in bottom_point_indexes])
results['top_actual'] = results['points_transformed'].apply(lambda x: [x[i] for i in top_point_indexes])
results['center_actual'] = results['points_transformed'].apply(lambda x: [x[i] for i in center_point_indexes])

# calculate mse for each subset
results['mse_bottom'] = results.apply(lambda x: compute_mse(x['bottom_predictions'], x['bottom_actual'], img_size), axis=1)
results['mse_top'] = results.apply(lambda x: compute_mse(x['top_predictions'], x['top_actual'], img_size), axis=1)
results['mse_center'] = results.apply(lambda x: compute_mse(x['center_predictions'], x['center_actual'], img_size), axis=1)

# calculate scores for each model
results['tp_bottom'], results['fp_bottom'], results['tn_bottom'], results['fn_bottom'] = zip(*results.apply(lambda x: compute_counts(x['bottom_predictions'], x['bottom_actual'], img_size), axis=1))
results['tp_top'], results['fp_top'], results['tn_top'], results['fn_top'] = zip(*results.apply(lambda x: compute_counts(x['top_predictions'], x['top_actual'], img_size), axis=1))
results['tp_center'], results['fp_center'], results['tn_center'], results['fn_center'] = zip(*results.apply(lambda x: compute_counts(x['center_predictions'], x['center_actual'], img_size), axis=1))

results.sample(5)

c:\Users\benev\Documents\Uni_Master\Masterarbeit\CODE\venv\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\benev\Documents\Uni_Master\Masterarbeit\CODE\venv\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


,points_transformed,positions,mse,subset,video,clip,frame,points,model,tp,...,tn_bottom,fn_bottom,tp_top,fp_top,tn_top,fn_top,tp_center,fp_center,tn_center,fn_center
150,"[[165.2, 68.8], [188.0, 68.8], [325.6, 68.8], ...","[(165, 68), (188, 68), (325, 68), (349, 68), (...",1.935903,CourtKeypoints,h9EjVAg1VzQ,Clip1,950,"{'top_left_corner': [413, 172], 'top_left_sing...",TN,15,...,0,0,7,0,0,0,1,0,0,0
113,"[[162.0, 82.8], [185.6, 82.8], [325.6, 83.2000...","[(161, 82), (185, 82), (325, 82), (348, 83), (...",2.900741,CourtKeypoints,VtqY_T2_gI4,Clip1,600,"{'top_left_corner': [405, 207], 'top_left_sing...",TN,15,...,0,0,7,0,0,0,1,0,0,0
146,"[[178.8, 62.8], [197.6, 62.8], [310.80002, 62....","[(178, 62), (197, 62), (309, 62), (328, 62), (...",2.077072,CourtKeypoints,eLWsgheI_OM,Clip1,250,"{'top_left_corner': [447, 157], 'top_left_sing...",TN,15,...,0,0,7,0,0,0,1,0,0,0
79,"[[-5.6, 73.6], [44.0, 72.8], [334.4, 68.4], [3...","[[-45.107677, 73.70964], [7.4556513, 73.52452]...",2372.479363,CourtKeypoints,M3IuThopUJU,Clip1,1200,"{'top_left_corner': [-14, 184], 'top_left_sing...",FC,0,...,0,5,0,5,0,2,0,1,0,0
110,"[[161.6, 76.8], [184.8, 76.8], [324.4, 76.8], ...","[[159.4634, 74.10581], [183.79211, 73.942566],...",45.823889,CourtKeypoints,V6NR0NAN_cA,Clip1,2350,"{'top_left_corner': [404, 192], 'top_left_sing...",FC,3,...,0,0,3,4,0,0,0,1,0,0


In [33]:
# compute metrcis
metrics_bottom = results.groupby('model')[['tp_bottom', 'fp_bottom', 'tn_bottom', 'fn_bottom']].sum().reset_index()
metrics_top = results.groupby('model')[['tp_top', 'fp_top', 'tn_top', 'fn_top']].sum().reset_index()
metrics_center = results.groupby('model')[['tp_center', 'fp_center', 'tn_center', 'fn_center']].sum().reset_index()

# calculate metrics
metrics_bottom[['accuracy', 'precision', 'recall', 'f1']] = metrics_bottom.apply(lambda x: pd.Series(calculate_metrics(x['tp_bottom'], x['fp_bottom'], x['tn_bottom'], x['fn_bottom'])), axis=1)
metrics_bottom['type'] = 'bottom'
metrics_bottom = metrics_bottom[['model', 'accuracy', 'precision', 'recall', 'f1', 'type']]

metrics_top[['accuracy', 'precision', 'recall', 'f1']] = metrics_top.apply(lambda x: pd.Series(calculate_metrics(x['tp_top'], x['fp_top'], x['tn_top'], x['fn_top'])), axis=1)
metrics_top['type'] = 'top'
metrics_top = metrics_top[['model', 'accuracy', 'precision', 'recall', 'f1', 'type']]

metrics_center[['accuracy', 'precision', 'recall', 'f1']] = metrics_center.apply(lambda x: pd.Series(calculate_metrics(x['tp_center'], x['fp_center'], x['tn_center'], x['fn_center'])), axis=1)
metrics_center['type'] = 'center'
metrics_center = metrics_center[['model', 'accuracy', 'precision', 'recall', 'f1', 'type']]

#combine
metrics_points = pd.concat([metrics_bottom, metrics_top, metrics_center])
#sort dataframe by model and type
metrics_points = metrics_points.sort_values(by=['model', 'type'])
metrics_points

,model,accuracy,precision,recall,f1,type
0,FC,0.193826,0.201649,0.822630,0.323901,bottom
0,FC,0.487437,0.487437,1.000000,0.655405,center
0,FC,0.288586,0.289001,0.995050,0.447911,top
1,TN,0.956210,0.991071,0.964518,0.977615,bottom
1,TN,0.989950,0.994949,0.994949,0.994949,center
1,TN,0.987078,0.992780,0.994215,0.993497,top


In [34]:
# compute sme
mse = results.groupby('model')[['mse', 'mse_bottom', 'mse_top', 'mse_center']].mean().reset_index()
mse

,model,mse,mse_bottom,mse_top,mse_center
0,FC,221.212804,299.938636,96.312434,59.052734
1,TN,7.912210,9.973426,6.631865,2.516099
